# Object-oriented scientific programming with C++

Matthias Möller, Jonas Thies, Cálin Georgescu, Jingya Li (Numerical Analysis, DIAM)

Lecture 7

## Goal of this lecture

- advanced data structures (<code>union</code>, <code>std::any</code>, <code>std::variant</code>, <code>std::tuple</code>)
- structuring C++ code into multiple files (build process: preprocessor, compiler, linker)
- building code with CMake

## Unions

C++ is a **strongly typed** programming language meaning that each object has a fixed type. Don't confuse this with *casting*, e.g., <code>int i = 1.6f;</code>. Variable <code>i</code> is of type <code>int</code> and the value <code>1.6f</code> is downcasted.

A <code>union</code> is a special class type that can hold one of its non-static data member at a time. 

**Example**:

In [1]:
union S {
    int   i;
    float f;
};

We can now create an object and assign a value to the first data member (<code>int i</code>):

In [12]:
#include <iostream>

S s{1234};
std::cout << s.i << std::endl;

1234


The content of the second data member is undefined/meaningless

In [11]:
std::cout << s.f << std::endl;

1.7292e-42


We can assign new values to the object <code>s</code> even for different data members

In [13]:
s.i = 4321; std::cout << s.i << std::endl;
s.f = 1.6f; std::cout << s.f << std::endl;

4321
1.6


A <code>union</code> always holds the value of the most recently assigned data member. Reading from another data member is undefined behavior.

Unions are a special class type with some **limitations** over regular classes and structures:

- a <code>union</code> can have member functions (including constructors and destructors), but not virtual functions.
- a <code>union</code> cannot have base classes and cannot be used as a base class.
- a <code>union</code> cannot have non-static data members of reference types.
- a <code>union</code> s at least as big as necessary to hold its largest data member.

## <code>std::variant</code>

In C++17 and later, the <code>std::variant</code> class is a type-safe alternative for a union.

**Example**:

In [1]:
#include <iostream>
#include <variant>

std::variant<int, float> v{1234};
std::cout << std::get<int>(v) << std::endl;
std::cout << std::get<float>(v) << std::endl;

1234


Standard Exception: std::get: wrong index for variant

In [2]:
v = 1.6f;
std::cout << std::get<float>(v) << std::endl;
std::cout << std::get<int>(v) << std::endl;

1.6


Standard Exception: std::get: wrong index for variant

Since <code>std::get&lt;TYPE&gt;</code> throws an exception if the wrong <code>TYPE</code> is requested one can catch inappropriate accesses with a <code>try-catch</code> construction.

Alternatively, one can use <code>std::get_if</code> which returns a pointer to the value of a pointed-to variant or null.

**Example**:

In [3]:
auto p = std::get_if<float>(&v);  std::cout << (p ? (*p) : 0) << std::endl;
auto q = std::get_if<int>(&v);    std::cout << (q ? (*q) : 0) << std::endl;

1.6
0


## <code>std::any</code>

A <code>union</code> or <code>std::variant</code> can only hold values of predefined type, i.e. it is not possible to hold *any* data type. Since C++17, the container class <code>std::any</code> can hold single values of *any* copy constructible type.

**Example**:

In [3]:
#include <iostream>
#include <any>

std::any a = 1234;
std::cout << a.type().name() << ": " << std::any_cast<int>(a) << std::endl;

a = 1.6f;
std::cout << a.type().name() << ": " << std::any_cast<float>(a) << std::endl;

i: 1234
f: 1.6


**Note**: An <code>std::any</code> object cannot be accessed without <code>std::any_cast</code>

In [4]:
std::cout << a << std::endl;

input_line_13:2:12: error: invalid operands to binary expression ('std::ostream' (aka 'basic_ostream<char>') and 'std::any')
 std::cout << a << std::endl;
 ~~~~~~~~~ ^  ~
/srv/conda/envs/notebook/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/10.4.0/ostream:245:7: note: candidate function not viable: no known conversion from 'std::any' to 'const void *' for 1st argument; take the address of the argument with &
      operator<<(const void* __p)
      ^
/srv/conda/envs/notebook/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/10.4.0/cstddef:125:5: note: candidate function template not viable: no known conversion from 'std::ostream' (aka 'basic_ostream<char>') to 'std::byte' for 1st argument
    operator<<(byte __b, _IntegerType __shift) noexcept
    ^
/srv/conda/envs/notebook/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/10.4.0/system_error:262:5: note: candidate function template not viable: no known conversion from 'std::any' to 'const std::error_code' for 2nd

Interpreter Error: 

## Separate compilation

C++ programs can vary greatly in size and complexity, as well as the number of people involved in the development. 
<br>

<center><img src='plots/lecture7-gismo.png' width="700px"></center>

## Single-file programs
<br>
<div style="border-style: double;border-width: 5px;border-color: RGB(252,169,133);" >
<h3 style="margin: auto; padding: 10px; color: RGB(252,169,133); ">Problem</h3>
    <div style="margin: auto;padding-left: 40px;padding-bottom: 20px;">
        <lu>
            <li>Many people work on the same project.</li>
            <li>A lot of source code.</li>
            <li>Need long time to compile.</li>
            <li>Want to share codes among different projects.</li>
        </lu>
    </div>
</div>

<br>
<div style="border-style: double;border-width: 5px;border-color: RGB(133,202,93);" >
<h3 style="margin: auto; padding: 10px; color: RGB(133,202,93); ">Solution: Multi-file programs</h3>
    <div style="margin: auto;padding-left: 40px;padding-bottom: 20px;">
        <lu>
            <li>Divide source code into seperate files.</li>
            <li>Compile files separately.</li>
            <li>Only recompile files when modified.</li>
        </lu>
    </div>
</div>

## Single-file programs vs. multi-file programs
<br>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;margin:0px auto;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-llyw{background-color:#c0c0c0;border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-llyw"></th>
    <th class="tg-llyw"><span style="font-weight:bold">Single-file programs</span></th>
    <th class="tg-llyw"><span style="font-weight:bold">Multi-file programs</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky"><span style="font-weight:bold">Suitability</span></td>
    <td class="tg-0pky">Ideal for <span style="font-weight:bold">small, simple projects</span><br>(Most assignments in this course)</td>
    <td class="tg-0pky"><span style="font-weight:bold">Large, complex projects</span></td>
  </tr>
  <tr>
    <td class="tg-0pky"><span style="font-weight:bold">Compilation Time</span></td>
    <td class="tg-0pky">Very long for large programs, <br>taking hours or days</td>
    <td class="tg-0pky">Much <span style="font-weight:bold">faster</span> as files are compiled <br>separately. Only changed files <br>need recompilation.</td>
  </tr>
  <tr>
    <td class="tg-0pky"><span style="font-weight:bold">Impact on Compiler</span></td>
    <td class="tg-0pky">Large single files might strain <br>or even break the compiler due<br> to resource demands.</td>
    <td class="tg-0pky">Less strain on the compiler as each <br>file is smaller and compiled <br>individually.</td>
  </tr>
  <tr>
    <td class="tg-0pky"><span style="font-weight:bold">Team Collaboration</span></td>
    <td class="tg-0pky">Difficult for team collaboration.<br>Team members might interfere with<br> each other's work.</td>
    <td class="tg-0pky">Facilitates <span style="font-weight:bold">parallel work</span>. <br>Team members can work on separate <br>files without interference, enhancing<br> productivity and minimizing conflicts.</td>
  </tr>
  <tr>
    <td class="tg-0pky"><span style="font-weight:bold">Modularity and Maintenance</span></td>
    <td class="tg-0pky">Low.</td>
    <td class="tg-0pky">High.</td>
  </tr>
  <tr>
    <td class="tg-0pky"><span style="font-weight:bold">Linking vs. Compilation Time</span></td>
    <td class="tg-0pky">Not applicable as there is only <br>one compilation step.</td>
    <td class="tg-0pky">Linking is required to combine <br>the separately compiled files into a <br>single executable. <span style="font-weight:bold">Linking is generally</span><br><span style="font-weight:bold"> much faster than compilation.</span></td>
  </tr>
</tbody>
</table>

## Seperate compilation
Every **_source code_** file is transformed through compilation into an **_object code_** file. These object code files are then **_linked together_** to create the final **_executable program_**.

**Source Code:** 
- Program in human-readable form (C++ language).

**Object Code:**
- Binary code: low-level representation of the source code, it's not yet a standalone program.
- Exact addresses of variables and functions not known, represented by symbols.


## Seperate compilation
**Linking:** 
- The linker takes all the object code files and combines them to create a single executable file.
- It resolves references to symbols that are defined in different object files or libraries.For example, if one object file has a function call to a function defined in another object file, the linker connects these two.

**Executable:** 
- Output of the linking process is the executable file, which can be run on a computer. 
- This file contains all the code and resources needed to execute the program.

## C++ build process
- **Headers** (*.h) + **Translation Units** (*.cpp) contain source code.
- **Preprocessor** performs text substitutions.
- **Compiler** translates translation units into object files.
- **Linker** links onject files and _external libraries_ into an executable.
<center><img src='plots/lecture7-buildmodel.png' width="700px"></center>

## C++ build process
<br>
Based on Weblab assignment
<center><img src='plots/lecture7-class_chart.png' width="900px"></center>

## Preprocessor \#
In the C++ build process, the preprocessor is the first step that runs before the actual compilation.
<center><img src='plots/lecture7-preprocessor.png' width="500px"></center>
<br>
<div style="border-style: double;border-width: 5px;border-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">The primary functions of the preprocessor include:</h3>
    <div style="margin: auto;padding-left: 40px;padding-bottom: 20px;">
        <lu>
            <li>modifying the source code</li>
            <li>processing preprocessor instructions (handling lines beginning with <code>#</code>)</li>
            <li>stripping out comments</li>
        </lu>
    </div>
</div>


## Preprocessor \#
<br>
<div style="border-style: double;border-width: 5px;border-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">Usage in C++ (and you should limit it to that)</h3>
    <div style="margin: auto;padding-left: 40px;padding-bottom: 20px;">
        <lu>
            <li>combining source code (<code>#include</code>)</li>
            <li>conditional compilation</li>
            <li>obtaining platform information during compilation</li>
        </lu>
    </div>
</div>

## Preprocessor MACROs
Macro: a powerful feature provided by the C++ preprocessor, a tool that processes your code before it is compiled. 

1. **Constant definitions**: Macros are often used to define constants. For example:
```C++
#define PI 3.14159
```
2. **Function-like macro:** Macros can also mimic functions. These are useful for small, repetitive code that doesn't need overhead of a function call. For instance:
```C++
#define SQUARE(x) ((x)*(x))
```
**NOTE:** it is important to use parentheses around parameters and the entire definition to ensure correct order of operations when the macro is used.


## Preprocessor MACROs
3. **Conditional compliation:** Macros can be used in conjunction with `#if`, `#ifdef`, `#ifndef`, and other preprocessor directives for conditional compilation.
```C++
#define DEBUG

#ifdef DEBUG
// Your own debug code goes there
#endif
```

4. **Platform-specific code:** They can be used to compile code conditionally for different platforms or compilers.
```C++
#ifdef _WIN32
// Windows-specific code
#endif
```

## Preprocessor MACROs 
5. And more... (but don't do it!)
<center><img src='plots/lecture7-preprocessormeme.jpg' width="400px"></center>

## Special MACROs
- `__FILE__` (current file name), `__LINE__` (current line number), `__DATE__`(MMM DD YYYY) and `__TIME__` (hh:mm:ss)
- `__cplusplus`: This macro is defined when a source code file is being compiled by the C++ compiler. Its value reflects the version of the C++ standard used by the compiler. 
    - C++98: 199711L
    - C++11: 201103L
    - C++14: 201402L
    - C++17: 201703L
    - C++20: 202002L
    
Example usage:
```C++
#if __cplusplus >= 201703L
    // C++17 (and later) code goes here
#endif
```

## <code>#include</code> directive
<code>#include</code>: used for including other files into the source file, typically header files.

There are two common usages:

-  Inserts a system header file from a location defined when the compiler was installed `#include <headerName>`. Example:
```C++
#include <iostream> // As we mentioned during lecture 1
```
- Inserts a file from the current directory. Example:
```C++
#include "filename"
```

<div style="display: flex; text-align:left; background-color: RGB(  191, 213, 232  );" >
<b>NOTE</b>: 
<ul>
    <li>Since header files are included in one or more <code>*.cpp</code> files using the <code>#include</code> directive, the content of these header files can be compiled multiple times -- once for each <code>*.cpp</code> that includes them. 
        </li>
    <li>Each <code>*.cpp</code> file is compiled only once to produce its corresaponding object file (<code>*.o</code>).    
        </li>
</ul>
</div>


## Problems with the <code>#include</code> directive
<br>
<center><img src='plots/lecture7-include_problem1.png' width="800px"></center>

## Problems with the <code>#include</code> directive continued

The definition of <code>square(int x)</code> is processed two times during the compilation of <code>main.cpp</code>.

<center><img src='plots/lecture7-include_problem2.png' width="800px"></center>

## Dealing with repeated <code>#include</code>s
The repeated inclusion of the same header file(s) can lead to **multiple definition errors** (conflicts during the linking stage) if not managed correctly.

To manage these issues, two common techniques can be used:
<div style="border-style: double;border-width: 5px;border-color: RGB(51,165,182);" >
    <div style="margin: auto;padding-left: 40px;padding-bottom: 20px;padding-top: 20px;">
        <lu>
            <li>Include guards</li>
            <li><code>#pragma once</code></li>
        </lu>
    </div>
</div>


## Include guards
<center><img src='plots/lecture7-include_problem3.png' width="700px"></center>

## <code>#pragma</code> directive
<code>#pragma</code>: a special preprocessor directive used to provide additional instructions to the compiler. 

It is easier to use and less error-prone compared to traditional include guards.

```C++
#pragma once // tell the compiler to include a file only once in a single compilation
```

Other usage:
```C++
#pragma warning(disable: 4507) // disables a specific warning with the number 4507
#pragma warning(default: 4507) // re-enable the warning
```

## Review: Declarations and definitions

A *declaration* in C++ serves to introduce or reiterate the name of an entity in the program.
```C++
extern int globalVar; // Declaration of a variable
void printMessage();  // Declaration of a function
class MyClass;        // Forward declaration of a class
```

A *definition* in C++ goes beyond the declaration by not only **introducing or reiterating the name and specifying the type** but also by **providing the actual implementation or value and allocating storage**.

```C++
int globalVar = 10;        // Definition of a variable
void printMessage() {      // Definition of a function
    std::cout << "Hello";
}
class MyClass {            // Definition of a class
public:
    void doSomething() {}
};
```

## Organizing Decls and Defs into Files

Simple example:
<center><img src='plots/lecture7-decl_def.png' width="900px"></center>

## What's in header?
Header files are intended to contain:
- **Function declarations**: declare the functions you intend to use or implement. This tells the compiler about their existence and how they should be called.
- **Class declarations**: declare the classes you intend to use of implement. This tells the compiler about their existence and how they should be instantiated.
- **Templates**: since templates need to be known at compile time in every file where they are used, they are usually defined in header files.
- **Global constants and macros**: define constants and macros that are to be shared across multiple source files.
- **Inline functions**: small functions that benefit from being inline (like getters and setters) can be defined in header files.

## Best practices
- **Include guards:** always use include guards (`#ifndef`, `#define`, `#endif`) or `#pragma once` in header files to prevent multiple inclusion issues.
- **Minimize dependencies:** include only what is **necessary** in header files to reduce compilation times and dependencies.
- **Consistent file naming:** keep a consistent naming scheme for your files. Typically, class names are used for the names of the corresponding header and source files.
- **Avoid using directives in headers**

## Namespace pollution
<br>
<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">Avoid in headers</h3>
</div>

- using namespaces
```C++
using namespace std;
```
- using symbols of a namespace
```C++
using std::cout;
```

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">WHY?</h3>
</div>

>It forces all symbols from the specified namespace into the global namespace for all source files that include that header. This can lead to unexpected name conflicts and **namespace pollution**.

## Linking
<br>
<div style="border: 1px solid RGB(0,0,0);" ><code>$ g++ -c main.cpp -o main.o
$ g++ -c math_util.cpp -o math_util.o
$ g++ main.o math_util.o -o myexe
$ ./myexe</code>
</div> 
<br>
<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">Explanation:</h3>
</div> 

- Preprocessing and compiling `main.cpp` yields `main.o`
- Preprocessing and compiling `math_util.cpp` yields `math_util.o`
- Linking `math_util.o` and `main.o` yields executable `myexe`
- Run program `myexe`

## CMake: depart from the Weblab nutshell to the great universe
A solution to streamline the often tedious and complex task of managing linking in C++ projects. 

CMake is **not** a build system like [Unix Make](https://en.wikipedia.org/wiki/Make_(software)) but a **build system generator**. It provides a family of tools and a _domain-specific_ language (DSL) to describe what the build system should achieve, you can reuse the same CMake scripts to obtain native build systems on any platform.

<center><img src='plots/lecture7-cmakememe.png' width="700px"></center>

## Generate and Build
The process of generating a <code>Makefile</code> and compiling with CMake on a Linux platform is as follows:
<div style="border-style: double;border-width: 5px;border-color: RGB(51,165,182);" >
    <ul>
        <li>Write the configuration file <code>CMakeLists.txt</code>.</li>
        <li>Run the command <code>cmake path/to/your/CMakeLists.txt</code> to generate a <code>Makefile</code>.</li>
        <li>Run the command <code>make</code> to compile your project.</li>
        </ul>
</div>

<center><img src='plots/lecture7-cmake1.png' width="900px"></center>

## Hello CMake!
A minimal project:
```C++
#include <iostream>

int main(){
    std::cout << "Hello CMake!" << std::endl;
    return 0;
}
```
Assuming the working directory only contains ``main.cpp``. We need to add ``CMakeLists.txt``.
```cmake
# Set the requirement on minimum version of CMake
cmake_minimum_required(VERSION 3.9)

# Declare project and its programming language.
project(HelloCMake)
set(CMAKE_CXX_STANDARD 11)

# Create executable target and linking
add_executable(hellocmake main.cpp)
```


## Hello CMake!
Now we are ready to call CMake and generate the <code>Makefile</code>: 
```bash
cmake .
```
`.` represents the current directory.

<br>
<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">Best practice: out-of-source build</h3>
</div> 

By specifying the project source root (**-S** option) and target build location (**-B** option) on the command line:
```bash
cmake -S . -B build/
```

## Hello CMake!
<br>
<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">Classic Approach</h3>
</div> 

The older CMake approach was to change to the build folder to explicitly run the build tool (**make**) from that folder:
```bash
mkdir build
cd build
cmake ..
make
```
The output should be look like this:

```bash
Scanning dependencies of target hellocmake
[ 50%] Building CXX object CMakeFiles/hellocmake.dir/main.cpp.o
[100%] Linking CXX executable hellocmake
[100%] Build target hellocmake
```

## END ...
<br>
<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">For more information about CMake</h3>
</div>

CMake – The Dark Arts: https://blog.feabhas.com/2021/07/cmake-part-1-the-dark-arts/

CMake Documentation: https://cmake.org/documentation/

CMake hands-on workshop: https://enccs.github.io/cmake-workshop/

C++ Starter Project with Complete CMake Setup by Jason Turner: https://github.com/cpp-best-practices/cmake_template
<br>
<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 10px; color: RGB(51,165,182); ">For more information about C++ program structure</h3>
</div>

Back to Basics: Compiling and Linking - Ben Saks: https://www.youtube.com/watch?v=cpkDQaYttR4

Beginner's Guide to Linkers: https://www.lurklurk.org/linkers/linkers.html
